#### Домашнее задание по теме «Рекомендации на основе содержания»

* Использовать dataset MovieLens
* Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
* Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
links = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\links.csv')
movies = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\movies.csv')
ratings = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\ratings.csv')
tags = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\tags.csv')

In [3]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [7]:
ratings[ratings.movieId==1]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
516,5,1,4.0,847434962
874,7,1,4.5,1106635946
1434,15,1,2.5,1510577970
1667,17,1,4.5,1305696483
...,...,...,...,...
97364,606,1,2.5,1349082950
98479,607,1,4.0,964744033
98666,608,1,2.5,1117408267
99497,609,1,3.0,847221025


In [8]:
data1 = movies.join(tags.set_index('movieId'), on='movieId')    # movies.merge(tags)
data1

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaN
9739,193585,Flint (2017),Drama,NaN,NaN,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaN


In [9]:
data2 = data1.join(ratings.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')
# data1.merge(ratings.set_index('movieId'), on='movieId')
data2

,movieId,title,genres,userId_left,tag,timestamp_left,userId_right,rating,timestamp_right
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,1.0,4.0,9.649827e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,5.0,4.0,8.474350e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,7.0,4.5,1.106636e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,15.0,2.5,1.510578e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaN,184.0,4.0,1.537109e+09
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaN,184.0,3.5,1.537110e+09
9739,193585,Flint (2017),Drama,NaN,NaN,NaN,184.0,3.5,1.537110e+09
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaN,184.0,3.5,1.537110e+09


In [10]:
data = data2[['movieId', 'genres', 'userId_left', 'tag', 'userId_right', 'rating']]
data

,movieId,genres,userId_left,tag,userId_right,rating
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5
...,...,...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,NaN,NaN,184.0,4.0
9738,193583,Animation|Comedy|Fantasy,NaN,NaN,184.0,3.5
9739,193585,Drama,NaN,NaN,184.0,3.5
9740,193587,Action|Animation,NaN,NaN,184.0,3.5


In [11]:
data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|')))
data

<ipython-input-11-0ef3973eb410>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|')))


,movieId,genres,userId_left,tag,userId_right,rating,Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy
...,...,...,...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,NaN,NaN,184.0,4.0,Action Animation Comedy Fantasy
9738,193583,Animation|Comedy|Fantasy,NaN,NaN,184.0,3.5,Animation Comedy Fantasy
9739,193585,Drama,NaN,NaN,184.0,3.5,Drama
9740,193587,Action|Animation,NaN,NaN,184.0,3.5,Action Animation


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285783 entries, 0 to 9741
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       285783 non-null  int64  
 1   genres        285783 non-null  object 
 2   userId_left   233234 non-null  float64
 3   tag           233234 non-null  object 
 4   userId_right  285762 non-null  float64
 5   rating        285762 non-null  float64
 6   Genre         285783 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 17.4+ MB


In [13]:
data.dropna(inplace=True)

<ipython-input-13-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 9732
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       233213 non-null  int64  
 1   genres        233213 non-null  object 
 2   userId_left   233213 non-null  float64
 3   tag           233213 non-null  object 
 4   userId_right  233213 non-null  float64
 5   rating        233213 non-null  float64
 6   Genre         233213 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 14.2+ MB


##### Выведем метрику на каждый фильм и добавим его в фичи

In [15]:
# посчитаем среднюю оценку на каждый фильм
mean_score_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    mean_score_mov[title] = group.rating.mean()

In [16]:
# посчитаем по каждому фильму количество рейтингов
num_action_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    num_action_mov[title] = group.shape[0]

In [17]:
# метрики
min_num_action_mov = np.min([num_action_mov[f] for f in num_action_mov.keys()])
max_num_action_mov = np.max([num_action_mov[f] for f in num_action_mov.keys()])
mean_num_action_mov = np.mean([num_action_mov[f] for f in num_action_mov.keys()])

In [18]:
# посчитаем метрику фильма
score_mov = {}

for f in num_action_mov.keys():
        score_mov[f] = (mean_score_mov[f] * (num_action_mov[f] - mean_num_action_mov)/
                        (max_num_action_mov - min_num_action_mov))

In [19]:
# добавим признак - метрику фильма

data['score_mov'] = data['movieId'].apply(lambda x: score_mov[x])
data

<ipython-input-19-55135121a7ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score_mov'] = data['movieId'].apply(lambda x: score_mov[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924
...,...,...,...,...,...,...,...,...
9710,187595,Action|Adventure|Children|Sci-Fi,62.0,star wars,586.0,5.0,Action Adventure Children SciFi,-0.009831
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,anime,184.0,3.5,Action Animation Comedy SciFi,-0.009201
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,comedy,184.0,3.5,Action Animation Comedy SciFi,-0.009201
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,gintama,184.0,3.5,Action Animation Comedy SciFi,-0.009201


##### Посчитаем среднюю оценку каждого userId и добавим в фичи

In [20]:
mean_mark = {}

for title, group in tqdm(data.groupby('userId_right')):
    mean_mark[title] = group.rating.mean()

In [21]:
data['mean_mark'] = data['userId_right'].apply(lambda x: mean_mark[x])
data

<ipython-input-21-ccece6ee142b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mean_mark'] = data['userId_right'].apply(lambda x: mean_mark[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404
...,...,...,...,...,...,...,...,...,...
9710,187595,Action|Adventure|Children|Sci-Fi,62.0,star wars,586.0,5.0,Action Adventure Children SciFi,-0.009831,4.421053
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,anime,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,comedy,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,gintama,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770


##### tfidf tag

In [22]:
Tag = {}
for title, group in tqdm(data.groupby('movieId')):
    Tag[title] = list(group.tag.unique())
Tag

{1: ['pixar', 'fun'],
 2: ['fantasy', 'magic board game', 'Robin Williams', 'game'],
 3: ['moldy', 'old'],
 5: ['pregnancy', 'remake'],
 7: ['remake'],
 11: ['politics', 'president'],
 14: ['politics', 'president'],
 16: ['Mafia'],
 17: ['Jane Austen'],
 21: ['Hollywood'],
 22: ['serial killer'],
 25: ['alcoholism'],
 26: ['Shakespeare'],
 28: ['In Netflix queue', 'Jane Austen'],
 29: ['kidnapping'],
 31: ['high school', 'teacher'],
 32: ['time travel',
  'Brad Pitt',
  'Bruce Willis',
  'mindfuck',
  'Post apocalyptic',
  'post-apocalyptic',
  'remake',
  'twist ending'],
 34: ['Animal movie',
  'pigs',
  'villain nonexistent or not needed for good story'],
 36: ['death penalty', 'Nun'],
 38: ['twins'],
 39: ['chick flick',
  'funny',
  'Paul Rudd',
  'quotable',
  'seen more than once',
  'Emma',
  'Jane Austen'],
 40: ['In Netflix queue', 'South Africa'],
 41: ['Shakespeare'],
 43: ['England'],
 45: ['Journalism'],
 46: ['wedding'],
 47: ['mystery', 'twist ending', 'serial killer'],

In [23]:
tfidf = TfidfVectorizer()

Tag2 = []
for i, j in Tag.items():
    #Tag2.append(re.sub("'", '', str(j).strip("['']")))
    Tag2.append(str(j).strip("['']"))
Tag2

tfidf.fit(Tag2)

tfidf_Tag = {}

for title, group in tqdm(Tag.items()):
    tfidf_Tag[title] = tfidf.transform(group).toarray()
tfidf_Tag

{1: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 2: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 3: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 5: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 7: array([[0., 0., 0., ..., 0., 0., 0.]]),
 11: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 14: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 16: array([[0., 0., 0., ..., 0., 0., 0.]]),
 17: array([[0., 0., 0., ..., 0., 0., 0.]]),
 21: array([[0., 0., 0., ..., 0., 0., 0.]]),
 22: array([[0., 0., 0., ..., 0., 0., 0.]]),
 25: array([[0., 0., 0., ..., 0., 0., 0.]]),
 26: array([[0., 0., 0., ..., 0., 0., 0.]]),
 28: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 29: array([[0., 0., 0.,

In [24]:
data['tfidf_Tag']= data['movieId'].apply(lambda x: tfidf_Tag[x])
data

<ipython-input-24-80fb4f71dd4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_Tag']= data['movieId'].apply(lambda x: tfidf_Tag[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark,tfidf_Tag
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...
9710,187595,Action|Adventure|Children|Sci-Fi,62.0,star wars,586.0,5.0,Action Adventure Children SciFi,-0.009831,4.421053,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,anime,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,comedy,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,gintama,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


##### tfidf Genre

In [25]:
Genre2 = {}
for title, group in tqdm(data.groupby('movieId')):
    Genre2[title] = list(group.Genre.unique())
Genre2

{1: ['Adventure Animation Children Comedy Fantasy'],
 2: ['Adventure Children Fantasy'],
 3: ['Comedy Romance'],
 5: ['Comedy'],
 7: ['Comedy Romance'],
 11: ['Comedy Drama Romance'],
 14: ['Drama'],
 16: ['Crime Drama'],
 17: ['Drama Romance'],
 21: ['Comedy Crime Thriller'],
 22: ['Crime Drama Horror Mystery Thriller'],
 25: ['Drama Romance'],
 26: ['Drama'],
 28: ['Drama Romance'],
 29: ['Adventure Drama Fantasy Mystery SciFi'],
 31: ['Drama'],
 32: ['Mystery SciFi Thriller'],
 34: ['Children Drama'],
 36: ['Crime Drama'],
 38: ['Children Comedy'],
 39: ['Comedy Romance'],
 40: ['Drama'],
 41: ['Drama War'],
 43: ['Drama'],
 45: ['Comedy Drama Thriller'],
 46: ['Drama Romance'],
 47: ['Mystery Thriller'],
 50: ['Crime Mystery Thriller'],
 52: ['Comedy Drama Romance'],
 58: ['Comedy Drama Romance'],
 62: ['Drama'],
 92: ['Drama Horror Thriller'],
 96: ['Comedy Drama'],
 101: ['Adventure Comedy Crime Romance'],
 104: ['Comedy'],
 107: ['Adventure Children Comedy Musical'],
 110: ['Act

In [26]:
Genre22 = []
for i, j in Genre2.items():
    Genre22.append(str(j).strip("['']"))

tfidf.fit(Genre22)

tfidf_Genre = {}

for title, group in tqdm(Genre2.items()):
    tfidf_Genre[title] = tfidf.transform(group).toarray()
tfidf_Genre

{1: array([[0.        , 0.3983147 , 0.52247057, 0.51053891, 0.28185923,
         0.        , 0.        , 0.        , 0.47778164, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]),
 2: array([[0.        , 0.49497089, 0.        , 0.63442775, 0.        ,
         0.        , 0.        , 0.        , 0.59372152, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]),
 3: array([[0.        , 0.        , 0.        , 0.        , 0.64167385,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.76697762, 0.        , 0.        , 0.        , 0.        ]]),
 5: array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]]),
 7: array([[0.        , 0.        , 0.        , 0.

In [27]:
data['tfidf_Genre']= data['movieId'].apply(lambda x: tfidf_Genre[x])
data

<ipython-input-27-6667e8883498>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_Genre']= data['movieId'].apply(lambda x: tfidf_Genre[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark,tfidf_Tag,tfidf_Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492..."
...,...,...,...,...,...,...,...,...,...,...,...
9710,187595,Action|Adventure|Children|Sci-Fi,62.0,star wars,586.0,5.0,Action Adventure Children SciFi,-0.009831,4.421053,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.4436120331508421, 0.45651642953482935, 0.0..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,anime,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.46448563209483706, 0.0, 0.6269903834182615..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,comedy,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.46448563209483706, 0.0, 0.6269903834182615..."
9732,193565,Action|Animation|Comedy|Sci-Fi,184.0,gintama,184.0,3.5,Action Animation Comedy SciFi,-0.009201,4.073770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.46448563209483706, 0.0, 0.6269903834182615..."


In [28]:
#data['Tag'] = data['movieId'].apply(lambda x: str(Tag[x]).strip("['']"))
#data['Tag'] = data['Tag'].apply(lambda x: re.sub("'", '', x))

#data

In [29]:
#tfidf = TfidfVectorizer()

#tfidf.fit(data['Tag'])
#data['tfidf_tag'] = data['Tag'].apply(lambda x: tfidf.fit(data['Tag']).transform(list(x)))

#data

In [30]:
#tfidf.fit(data['Genre'])
#data['tfidf_Genre'] = data['Genre'].apply(lambda x: tfidf.fit(data['Genre']).transform(x))

#data

In [36]:
X = data[data['userId_right']==1][['movieId', 'tfidf_Tag', 'tfidf_Genre', 'score_mov', 'mean_mark']]
y = data[data['userId_right']==1]['rating']

In [37]:
X

,movieId,tfidf_Tag,tfidf_Genre,score_mov,mean_mark
0,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492...",0.034924,4.040472
0,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492...",0.034924,4.040472
0,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.39831469859732216, 0.5224705725321492...",0.034924,4.040472
2,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.641673854111799, 0.0, ...",-0.002703,4.040472
2,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.641673854111799, 0.0, ...",-0.002703,4.040472
...,...,...,...,...,...
2836,3793,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5470389808892225, 0.5629520024471536, 0.0,...",0.043135,4.040472
2836,3793,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5470389808892225, 0.5629520024471536, 0.0,...",0.043135,4.040472
2836,3793,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5470389808892225, 0.5629520024471536, 0.0,...",0.043135,4.040472
2836,3793,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5470389808892225, 0.5629520024471536, 0.0,...",0.043135,4.040472


In [33]:
lr = LinearRegression()

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
lr.fit(X_train, y_train)

ValueError: setting an array element with a sequence.